# Working with Data Sets


In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
Films=pd.read_csv('datasets\movies.csv', sep=',',decimal=".")

In [4]:
Films.head()

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description
0,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...
1,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...
2,3,Inception,2010,8.8,74.0,"Action, Adventure, Sci-Fi",1760209,Christopher Nolan,148,292.58,A thief who steals corporate secrets through t...
3,4,Fight Club,1999,8.8,66.0,Drama,1609459,David Fincher,139,37.03,An insomniac office worker and a devil-may-car...
4,5,Pulp Fiction,1994,8.9,94.0,"Crime, Drama",1570194,Quentin Tarantino,154,107.93,"The lives of two mob hitmen, a boxer, a gangst..."


In [5]:
Oscar=pd.read_csv('datasets/the_oscar_award.csv')

In [6]:
Oscar.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


### Первый данные с фильмами не достаточны для нас, так как там нет записи об актерах, которые нужны для оценки популярности фильма


In [160]:
fulltable = pd.merge(left=Films, right=Oscar[['year_film','year_ceremony','category','name', 'film','winner']], left_on= ['Title','Year'], right_on =['film','year_film'], how='left')
fulltable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13521 entries, 0 to 13520
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           13521 non-null  int64  
 1   Title          13521 non-null  object 
 2   Year           13521 non-null  int64  
 3   Score          13521 non-null  float64
 4   Metascore      9253 non-null   float64
 5   Genre          13521 non-null  object 
 6   Vote           13521 non-null  int64  
 7   Director       13520 non-null  object 
 8   Runtime        13521 non-null  int64  
 9   Revenue        10359 non-null  float64
 10  Description    13521 non-null  object 
 11  year_film      5030 non-null   float64
 12  year_ceremony  5030 non-null   float64
 13  category       5030 non-null   object 
 14  name           5030 non-null   object 
 15  film           5030 non-null   object 
 16  winner         5030 non-null   object 
dtypes: float64(5), int64(4), object(8)
memory usage: 1

In [161]:
# заменяем на 1 и 0 в таблице винера
d = {False : 0, True : 1}
fulltable['winner'] = fulltable['winner'].map(d)

#находим индексы строк которые нужно заполнить
all_indexes = fulltable[fulltable.winner.isnull()].index
fulltable.loc[all_indexes, 'year_film':'winner'] = -1


In [162]:
metascore_indexes = fulltable[fulltable.Metascore.isnull()].index
fulltable.loc[metascore_indexes, 'Metascore'] = -1
revenue_indexes = fulltable[fulltable.Revenue.isnull()].index
fulltable.loc[revenue_indexes, 'Revenue'] = -1
director_indexes = fulltable[fulltable.Director.isnull()].index
fulltable.loc[director_indexes, 'Director'] = 'No info'
fulltable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13521 entries, 0 to 13520
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           13521 non-null  int64  
 1   Title          13521 non-null  object 
 2   Year           13521 non-null  int64  
 3   Score          13521 non-null  float64
 4   Metascore      13521 non-null  float64
 5   Genre          13521 non-null  object 
 6   Vote           13521 non-null  int64  
 7   Director       13521 non-null  object 
 8   Runtime        13521 non-null  int64  
 9   Revenue        13521 non-null  float64
 10  Description    13521 non-null  object 
 11  year_film      13521 non-null  float64
 12  year_ceremony  13521 non-null  float64
 13  category       13521 non-null  object 
 14  name           13521 non-null  object 
 15  film           13521 non-null  object 
 16  winner         13521 non-null  float64
dtypes: float64(6), int64(4), object(7)
memory usage: 2

In [172]:
newtable=fulltable.sort_values(by=['Title', 'winner'], ascending=[True, False])
newtable[newtable['Title'] == 'The Dark Knight']

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description,year_film,year_ceremony,category,name,film,winner
7,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,ACTOR IN A SUPPORTING ROLE,Heath Ledger,The Dark Knight,1.0
12,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,SOUND EDITING,Richard King,The Dark Knight,1.0
8,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,ART DIRECTION,Art Direction: Nathan Crowley; Set Decoration:...,The Dark Knight,0.0
9,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,CINEMATOGRAPHY,Wally Pfister,The Dark Knight,0.0
10,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,FILM EDITING,Lee Smith,The Dark Knight,0.0
11,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,MAKEUP,"John Caglione, Jr. and Conor O'Sullivan",The Dark Knight,0.0
13,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,SOUND MIXING,"Lora Hirschberg, Gary Rizzo and Ed Novick",The Dark Knight,0.0
14,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,VISUAL EFFECTS,"Nick Davis, Chris Corbould, Tim Webber and Pau...",The Dark Knight,0.0


In [175]:
newtable.drop_duplicates(subset ="Title", 
                     keep = 'first', inplace = True)
newtable[newtable['Title'] == 'The Dark Knight']

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description,year_film,year_ceremony,category,name,film,winner
7,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...,2008.0,2009.0,ACTOR IN A SUPPORTING ROLE,Heath Ledger,The Dark Knight,1.0


In [178]:
newtable[newtable['Title'] == 'Lion']

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description,year_film,year_ceremony,category,name,film,winner
2169,935,Lion,2016,8.1,69.0,"Biography, Drama",167997,Garth Davis,118,51.69,A five-year-old Indian boy gets lost on the st...,2016.0,2017.0,ACTOR IN A SUPPORTING ROLE,Dev Patel,Lion,0.0


In [230]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(newtable, test_size=0.4)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3880 entries, 7935 to 4918
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           3880 non-null   int64  
 1   Title          3880 non-null   object 
 2   Year           3880 non-null   int64  
 3   Score          3880 non-null   float64
 4   Metascore      3880 non-null   float64
 5   Genre          3880 non-null   object 
 6   Vote           3880 non-null   int64  
 7   Director       3880 non-null   object 
 8   Runtime        3880 non-null   int64  
 9   Revenue        3880 non-null   float64
 10  Description    3880 non-null   object 
 11  year_film      3880 non-null   float64
 12  year_ceremony  3880 non-null   float64
 13  category       3880 non-null   object 
 14  name           3880 non-null   object 
 15  film           3880 non-null   object 
 16  winner         3880 non-null   float64
dtypes: float64(6), int64(4), object(7)
memory usage: 

In [231]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
X=train[['Score', 'Revenue', 'Vote']]
y=train.winner
#X=train['Score']
#y=train.winner
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [232]:
expected = test.winner
predicted = model.predict(test[['Score', 'Revenue', 'Vote']])
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted,labels=[1,0,-1]))

              precision    recall  f1-score   support

        -1.0       0.85      0.99      0.92      3286
         0.0       0.32      0.02      0.03       379
         1.0       0.42      0.06      0.11       215

    accuracy                           0.85      3880
   macro avg       0.53      0.36      0.35      3880
weighted avg       0.78      0.85      0.79      3880

[[  13    6  196]
 [   8    6  365]
 [  10    7 3269]]
